In [80]:
import os
import pickle
import numpy as np
import cupy as cp
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import scipy.signal as scisig
import scipy.stats
from scipy.interpolate import interp1d 
from collections import Counter
import scipy.interpolate as interp
import cupyx.scipy.signal as cusig
import neurokit2 as nk
from scipy.signal import find_peaks, welch, butter, filtfilt
from scipy.optimize import curve_fit
import cupyx.scipy.signal as cpysig
import gc

In [81]:
import faulthandler
faulthandler.enable()

In [82]:
fs_dict = {'PPG': 1000, 'EDA': 1000, 'RESP': 1000, 'ECG': 1000, 'label': 1000}
WINDOW_IN_SECONDS = 30
STRIDE_IN_SECONDS = 0.75
label_dict = {'baseline': 1, 'stress': 2, 'cognitive load': 3}
int_to_label = {1: 'baseline', 2: 'stress', 3: 'cognitive load'}
feat_names = None
DATA_PATH = r'C:\Users\IALAB\Downloads\WESAD_TEST\dataProcessed/'
SAVE_PATH = r'C:\Users\IALAB\Downloads\WESAD_TEST\features_30_075/'


In [83]:
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

In [84]:
#def eda_stats(y):
#    Fs = fs_dict['EDA']
#    yn = (y - y.mean()) / y.std()
#    print(yn)
#    print("calculating eda stats")
#    [r, p, t, l, d, e, obj] = cvxEDA.cvxEDA(yn, 1. / Fs)
#    return [r, p, t, l, d, e, obj]

In [85]:
def butter_filter(data, cutoff, fs, order=4, filter_type='low'):
    """
    Aplica un filtro Butterworth a los datos.

    Parámetros:
    - data: array de la señal a filtrar.
    - cutoff: frecuencia de corte (o tupla en caso de bandpass).
    - fs: frecuencia de muestreo.
    - order: orden del filtro.
    - filter_type: 'low', 'high' o 'band'.

    Retorna:
    - Señal filtrada.
    """
    nyq = 0.5 * fs  # Frecuencia de Nyquist

    # Normalizar la frecuencia de corte
    if isinstance(cutoff, (list, tuple)):  # Band-pass o Band-stop
        normal_cutoff = [c / nyq for c in cutoff]
    else:  # Low-pass o High-pass
        normal_cutoff = cutoff / nyq

    # Crear el filtro Butterworth
    b, a = scisig.butter(order, normal_cutoff, btype=filter_type, analog=False)

    # Aplicar el filtro con filtfilt para evitar desfases
    return scisig.filtfilt(b, a, np.array(data))


def get_slope(signal):
    if len(signal) < 2:  # Evita divisiones por 0 o valores no válidos
        return 0
    
    # Convertir a pandas.Series si es necesario
    if not isinstance(signal, pd.Series):
        signal = pd.Series(signal)

    resu = (signal.iloc[-1] - signal.iloc[0]) / (len(signal) - 1)  # Evita dividir entre len(signal)

    # Convertir a escalar si es un array
    if isinstance(resu, np.ndarray):
        resu = resu.flatten()[0]
    
    return resu

def get_peak_freq(x, fs):
    f, Pxx = scisig.periodogram(x, fs=fs)
    psd_dict = {amp: freq for amp, freq in zip(Pxx, f)}
    return psd_dict[max(psd_dict.keys())]

def get_window_stats(data, label=-1):
    mean_features = np.mean(data)
    std_features = np.std(data)
    min_features = np.amin(data)
    max_features = np.amax(data)

    features = {'mean': mean_features, 'std': std_features, 'min': min_features, 'max': max_features,
                'label': label}
    return features


def get_net_accel(data):
    return (data['ACC_x'] ** 2 + data['ACC_y'] ** 2 + data['ACC_z'] ** 2).apply(lambda x: np.sqrt(x))


def get_peak_freq(x):
    f, Pxx = scisig.periodogram(x, fs=8)
    psd_dict = {amp: freq for amp, freq in zip(Pxx, f)}
    peak_freq = psd_dict[max(psd_dict.keys())]
    return peak_freq


# https://github.com/MITMediaLabAffectiveComputing/eda-explorer/blob/master/AccelerometerFeatureExtractionScript.py
def filterSignalFIR(eda, cutoff=0.4, numtaps=64):
    f = cutoff / (fs_dict['ACC'] / 2.0)
    FIR_coeff = scisig.firwin(numtaps, f)

    return scisig.lfilter(FIR_coeff, 1, eda.flatten())

def compute_hr(ecg_signal, fs, plot=False):
    # Detectar picos R
    peaks, _ = scisig.find_peaks(ecg_signal, distance=fs*0.6, height=np.mean(ecg_signal) + np.std(ecg_signal))
    
    # Calcular intervalos RR (diferencias entre picos)
    rr_intervals = np.diff(peaks) / fs  # Convertir a segundos
    
    # Calcular HR (latidos por minuto)
    hr_mean = 60 / np.mean(rr_intervals) if len(rr_intervals) > 0 else np.nan
    hr_std = np.std(60 / rr_intervals) if len(rr_intervals) > 0 else np.nan

    return hr_mean, hr_std

def triangle(x, a, b, c):
    return np.maximum(0, a - np.abs(x - b) / c)

In [86]:
def compute_eda_metrics(eda_signal, fs):
    eda_signal = np.array(eda_signal).flatten()
    if len(eda_signal) < 2:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    scl_mean = np.mean(eda_signal)
    scl_std = np.std(eda_signal)

    # Diferencia de la señal para detectar SCRs
    diff_signal = np.gradient(eda_signal) * 10

    # Detectar SCRs sin normalizar
    scr_peaks, properties = scisig.find_peaks(
        diff_signal, 
        height=np.std(diff_signal) * 1,  # Ajuste del umbral
        distance=fs*0.3  # Permitir detección más frecuente
    )

    scr_mean = np.mean(diff_signal[scr_peaks]) if scr_peaks.size > 0 else np.nan
    scr_std = np.std(diff_signal[scr_peaks]) if scr_peaks.size > 0 else np.nan
    scr_count = len(scr_peaks)
    scr_amp = np.max(np.abs(diff_signal[scr_peaks])) if scr_count > 0 else np.nan
    scr_sum = np.sum(diff_signal[scr_peaks]) if scr_count > 0 else np.nan
    scr_area = np.trapz(diff_signal[scr_peaks]) if scr_count > 1 else 0

    # Correlación de SCL con el tiempo
    time = np.arange(len(eda_signal)) / fs  # Convertir a segundos
    corr_SCL_t = np.corrcoef(time, eda_signal)[0, 1] if len(eda_signal) > 1 else np.nan
    corr_SCL_t = np.nan_to_num(corr_SCL_t)  # Evitar NaNs

    return scl_mean, scl_std, scr_mean, scr_std, corr_SCL_t, scr_count, scr_amp, scr_sum, scr_area

In [87]:
def compute_hrv_metrics(ecg_signal, fs):
    ecg_signal = np.array(ecg_signal).flatten()
    
    # Detección de picos R mejorada
    peaks, _ = scisig.find_peaks(ecg_signal, distance=fs*0.6, height=np.mean(ecg_signal) + np.std(ecg_signal))

    if len(peaks) < 2:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    # Cálculo de intervalos RR
    rr_intervals = np.diff(peaks) * (1000 / fs)  # Convertir a milisegundos
    rr_intervals = rr_intervals[(rr_intervals > 400) & (rr_intervals < 1500)]  # Filtro fisiológico

    if len(rr_intervals) < 2:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    # Features temporales
    nn50 = np.sum(np.abs(np.diff(rr_intervals)) > 50)
    pnn50 = (nn50 / len(rr_intervals)) * 100 if len(rr_intervals) > 0 else np.nan
    rms_hrv = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))

    # Cálculo de TINN
    hist, bin_edges = np.histogram(rr_intervals, bins='auto')
    tinn = bin_edges[np.argmax(hist)] if len(hist) > 0 else np.nan

    # Interpolación mejorada
    time_rr = np.cumsum(rr_intervals) / 1000
    time_rr = time_rr - time_rr[0]  # Normalizar inicio en 0

    fs_resample = 4.0  # Frecuencia de resampleo
    time_resampled = np.arange(0, time_rr[-1], 1 / fs_resample)
    interp_func = interp1d(time_rr, rr_intervals, kind="linear", fill_value="extrapolate")
    rr_resampled = interp_func(time_resampled)

    # Análisis espectral (PSD)
    fxx, pxx = scisig.welch(rr_resampled, fs=fs_resample, nperseg=min(len(rr_resampled), 256))
    lf_band, hf_band = (0.04, 0.15), (0.15, 0.4)

    lf_mask = (fxx >= lf_band[0]) & (fxx <= lf_band[1])
    hf_mask = (fxx >= hf_band[0]) & (fxx <= hf_band[1])

    lf = np.trapz(pxx[lf_mask], fxx[lf_mask]) if np.any(lf_mask) else np.nan
    hf = np.trapz(pxx[hf_mask], fxx[hf_mask]) if np.any(hf_mask) else np.nan
    lf_hf_ratio = lf / hf if (hf > 0 and not np.isnan(hf)) else np.nan

    sum_f = np.trapz(pxx, fxx)
    rel_f = (lf + hf) / sum_f if sum_f > 0 else np.nan
    lf_norm = (lf / (lf + hf)) * 100 if (lf + hf) > 0 else np.nan
    hf_norm = (hf / (lf + hf)) * 100 if (lf + hf) > 0 else np.nan

    return nn50, pnn50, tinn, rms_hrv, lf, hf, lf_hf_ratio, sum_f, rel_f, lf_norm, hf_norm

In [88]:
def compute_respiration_metrics(resp_signal, fs):
    resp_signal = np.array(resp_signal).flatten()
    peaks, _ = scisig.find_peaks(resp_signal, distance=fs * 0.3)
    troughs, _ = scisig.find_peaks(-resp_signal, distance=fs * 0.3)
    
    if len(peaks) < 2 or len(troughs) < 2:
        return 0, 0, 0, 0, 0, 0, 0, 0, 0

    inspiration_durations = np.diff(peaks) / fs
    expiration_durations = np.diff(troughs) / fs

    I_mean = np.mean(inspiration_durations)
    I_std = np.std(inspiration_durations)
    E_mean = np.mean(expiration_durations)
    E_std = np.std(expiration_durations)

    ie_ratio = I_mean / E_mean if E_mean > 0 else 0
    resp_range = np.max(resp_signal) - np.min(resp_signal)
    insp_vol = np.mean(resp_signal[peaks]) - np.mean(resp_signal[troughs])
    resp_rate = len(peaks) / (len(resp_signal) / fs)
    resp_duration = len(resp_signal) / fs

    return I_mean, I_std, E_mean, E_std, ie_ratio, resp_range, insp_vol, resp_rate, resp_duration

In [89]:
def compute_ppg_features(ppg_signal, fs, plot=False):
    ppg_signal = np.array(ppg_signal).flatten()

    if len(ppg_signal) < 2:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    # Detección de picos en PPG
    peak_indices, _ = scisig.find_peaks(ppg_signal, distance=fs*0.6, prominence=(np.std(ppg_signal) * 0.5, None))

    if len(peak_indices) < 2:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    ibi = np.diff(peak_indices) / fs
    hr = 60 / ibi

    hr_mean = np.mean(hr)
    hr_std = np.std(hr)
    rmssd = np.sqrt(np.mean(np.diff(ibi) ** 2))
    sdnn = np.std(ibi)

    # Frecuencia de muestreo para IBI
    fs_ibi = max(1 / np.mean(ibi), 0.5)  # No menor a 0.5 Hz

    # PSD con Welch si hay suficientes puntos
    if len(ibi) > 4:  
        fxx, pxx = scisig.welch(ibi, fs=fs_ibi, nperseg=min(128, len(ibi)))
    else:
        fxx, pxx = np.array([]), np.array([])

    # Cálculo de LF y HF solo si hay valores en la PSD
    if np.sum(pxx) > 0:
        lf_band = (fxx >= 0.04) & (fxx < 0.15)
        hf_band = (fxx >= 0.15) & (fxx < 0.4)

        lf = np.trapz(pxx[lf_band]) if np.any(lf_band) else np.nan
        hf = np.trapz(pxx[hf_band]) if np.any(hf_band) else np.nan
        lf_hf_ratio = lf / hf if hf > 0 else np.nan
    else:
        lf, hf, lf_hf_ratio = np.nan, np.nan, np.nan

    # Características adicionales
    amplitudes = ppg_signal[peak_indices]
    pav_mean = np.mean(amplitudes)
    pav_std = np.std(amplitudes)

    rise_time = np.mean(np.diff(peak_indices[:len(peak_indices)//2]) / fs)
    decay_time = np.mean(np.diff(peak_indices[len(peak_indices)//2:]) / fs)
    # Índice de reflexión (RI)
    
    amplitudes = ppg_signal[peak_indices]
    if len(amplitudes) > 1 and amplitudes[0] != 0:
        ri = amplitudes[-1] / amplitudes[0]
    else:
        ri = np.nan

    return hr_mean, hr_std, rmssd, sdnn, lf, hf, lf_hf_ratio, len(peak_indices), rise_time, decay_time, pav_mean, pav_std, ri

In [90]:
class SubjectData:
    def __init__(self, main_path, subject_number):
        self.name = f'{subject_number:03d}'  # Asegurar que el número tenga formato 000
        file_path = os.path.join(main_path, f"{self.name}*.csv")  # Ruta de los archivos CSV

        # Buscar el archivo correspondiente
        files = sorted([f for f in os.listdir(os.path.join(main_path)) if f.startswith(self.name)])
        if not files:
            raise FileNotFoundError(f"No se encontró archivo para el sujeto {self.name}")

        # Cargar el CSV en un DataFrame
        self.data = pd.read_csv(os.path.join(main_path, files[0]))

        # Extraer etiquetas
        self.labels = self.data['label']

    def get_data(self):
        # Retorna los datos fisiológicos excepto el tiempo y la etiqueta
        return self.data.drop(columns=['time', 'label'])

In [91]:
def compute_features(data_dict, fs_dict):
    feature_dict = {}
    # ECG y BVP

    ecg_signal = data_dict['ECG']
    eda_signal = data_dict['EDA']
    ppg_signal = data_dict['PPG']
    resp_signal = data_dict['RESP']

    
    hr_mean, hr_std = compute_hr(ecg_signal, fs_dict['ECG'])

    feature_dict['HR_mean'] = hr_mean
    feature_dict['HR_std'] = hr_std


    nn50, pNN50, tinn, rmsHRV, lf, hf, lf_hf, sum_f, rel_f, lf_norm, hf_norm = compute_hrv_metrics(ecg_signal, fs_dict['ECG'])

    feature_dict.update({
        'NN50': nn50, 'pNN50': pNN50, 'TINN': tinn, 
        'rmsHRV': rmsHRV, 'LF': lf, 'HF': hf, 'LF_HF': lf_hf,
        'sum_f': sum_f, 'rel_f': rel_f, 'LF_norm': lf_norm, 'HF_norm': hf_norm
    })


    hr_mean, hr_std, rmssd, sdnn, lf, hf, lf_hf, num_beats, rise_time, decay_time, pav_mean, pav_std, ri = compute_ppg_features(ppg_signal, fs_dict['PPG'])

    feature_dict.update({
        'PPG_HR_mean': hr_mean, 'PPG_HR_std': hr_std, 'PPG_RMSSD': rmssd, 'PPG_SDNN': sdnn,
        'PPG_LF': lf, 'PPG_HF': hf, 'PPG_LF_HF': lf_hf, 'PPG_num_beats': num_beats,
        'PPG_RiseTime': rise_time, 'PPG_DecayTime': decay_time, 
        'PPG_PAV_mean': pav_mean, 'PPG_PAV_std': pav_std, 'PPG_RI': ri
    })

    feature_dict.update({
        'EDA_mean': np.mean(eda_signal), 'EDA_std': np.std(eda_signal),
        'EDA_min': np.min(eda_signal), 'EDA_max': np.max(eda_signal),
        'EDA_range': np.max(eda_signal) - np.min(eda_signal), 'EDA_slope': get_slope(eda_signal)
    })



    scl_mean, scl_std, scr_mean, scr_std, corr_scl_t, scr_count, scr_amp, scr_sum, scr_area = compute_eda_metrics(eda_signal, fs_dict['EDA'])

    feature_dict.update({
        'scl_mean': scl_mean, 'scl_std': scl_std, 'scr_mean': scr_mean, 'scr_std': scr_std,
        'corr_scl_t': corr_scl_t, 'scr_count': scr_count, 'scr_amp': scr_amp,
        'scr_sum': scr_sum, 'scr_area': scr_area
    })

    feature_dict.update({
        'Resp_mean': np.mean(resp_signal), 'Resp_std': np.std(resp_signal),
    })

    I_mean, I_std, E_mean, E_std, ie_ratio, resp_range, insp_vol, resp_rate, resp_duration = compute_respiration_metrics(resp_signal, fs_dict['RESP'])
              
    feature_dict.update({
        'Resp_I_mean': I_mean, 'Resp_I_std': I_std, 'Resp_E_mean': E_mean, 'Resp_E_std': E_std,
        'Resp_IE_ratio': ie_ratio, 'Resp_range': resp_range, 'Resp_insp_vol': insp_vol,
        'Resp_rate': resp_rate, 'Resp_duration': resp_duration
    })

    # Convertir a DataFrame con solo una fila
    df = pd.DataFrame([feature_dict])

    df["EDA_slope"] = df["EDA_slope"].apply(lambda x: x[0] if isinstance(x, list) else x)

    return df

In [92]:
def get_samples(data_dict, labels, fs_dict, stride_seconds):
    global feat_names
    global WINDOW_IN_SECONDS

    samples = []
    all_samples = []
    
    # Convertir tiempo a muestras
    window_len = int(fs_dict['ECG'] * WINDOW_IN_SECONDS)  # Se toma una señal como referencia
    stride_len = int(fs_dict['ECG'] * stride_seconds)  

    num_ventanas = (len(labels) - window_len) // stride_len + 1
    print(f"El número de ventanas esperadas es: {num_ventanas}")

    last_progress = -10
    processed = 0

    all_samples = pd.DataFrame()

    for start in range(0, len(labels) - window_len + 1, stride_len):
        end = start + window_len

        processed += 1  # Incrementar contador manualmente
        progress = processed / num_ventanas * 100

        if processed % 500 == 0:
            gc.collect()
            print(f"Recolección de basura ejecutada en ventana {processed}.")

        if processed % 50 == 0:

            print(f"\rProgreso: {progress:.4f}% completado", end="", flush=True)

        # Extraer ventana de cada señal
        print(f"\nVentana {start} - {end}", end="", flush= True)
        window_data = {key: val[start:end] for key, val in data_dict.items()}
        window_labels = labels[start:end]  # Extraer etiquetas de la ventana

        # Aplicar hard labeling: etiqueta más frecuente en la ventana
        label_counts = Counter(window_labels)
        most_common_labels = label_counts.most_common()  # [(label1, count1), (label2, count2), ...]
        
        # Si hay empate, tomar la primera que aparece en la ventana original
        max_count = most_common_labels[0][1]
        candidate_labels = [label for label, count in most_common_labels if count == max_count]
        chosen_label = next(label for label in window_labels if label in candidate_labels)

        features_df = compute_features(window_data, fs_dict)

        # Agregar la etiqueta al DataFrame de features
        features_df['label'] = chosen_label

        all_samples = pd.concat([all_samples, features_df], ignore_index=True)

    if all_samples.empty:
        print("Advertencia: No se generaron muestras en get_samples(), devolviendo DataFrame vacío.")

    print("\n Procesamiento de ventanas completado.")  
    
    return all_samples

In [93]:
def make_patient_data(subject_id):
    global SAVE_PATH
    global WINDOW_IN_SECONDS

    # Crear objeto SubjectData
    subject = SubjectData(main_path=DATA_PATH, subject_number=subject_id)

    # Obtener datos del pecho (ahora en DataFrame)
    data_df = subject.get_data()

    print("Columnas disponibles:", data_df.columns)
    print("Etiquetas:", subject.labels.value_counts())

    # Definir clases de interés
    valid_labels = {1, 2, 3}
    
    # Filtrar datos válidos
    mask = subject.labels.isin(valid_labels)
    filtered_labels = subject.labels[mask].reset_index(drop=True)
    filtered_data = data_df[mask].reset_index(drop=True)

    filtered_data = filtered_data.copy()

    filtered_data['ECG'] = butter_filter(filtered_data['ECG'], cutoff=(0.5, 40), fs=fs_dict['ECG'], order=4, filter_type='band')
    filtered_data['PPG'] = butter_filter(filtered_data['PPG'], cutoff=(0.5, 8), fs=fs_dict['PPG'], order=2, filter_type='band')
    filtered_data['EDA'] = butter_filter(filtered_data['EDA'], cutoff=1, fs=fs_dict['EDA'], order=4, filter_type='low')
    filtered_data['RESP'] = butter_filter(filtered_data['RESP'], cutoff=0.5, fs=fs_dict['RESP'], order=4, filter_type='low')

    print("Data lista para procesar: " + str(len(filtered_labels)))

    # Calcular características
    samples = get_samples(filtered_data, filtered_labels, fs_dict=fs_dict, stride_seconds=STRIDE_IN_SECONDS)

    print("Características calculadas")

    if not isinstance(samples, pd.DataFrame):
        all_samples = pd.concat(samples, ignore_index=True)
    else:
        all_samples = samples.copy()

    all_samples['label'] = all_samples['label'].astype(int)
    all_samples = pd.concat([all_samples.drop('label', axis=1), pd.get_dummies(all_samples['label'])], axis=1)

    # Guardar como CSV
    all_samples.to_csv(f'{SAVE_PATH}/{subject_id}_features.csv', index=False)

    # **Liberar memoria**
    del subject, all_samples, samples, data_df, filtered_data, filtered_labels
    gc.collect()  # Forzar recolección de basura

In [94]:
def combine_files(subjects):
    df_list = []
    for s in subjects:
        df = pd.read_csv(f'{SAVE_PATH}/{s}_features.csv')
        df['subject'] = s
        df_list.append(df)

    df = pd.concat(df_list)

    print(df.head(10))
    print(df.columns)

    df[['1', '2', '3']] = df[['1', '2', '3']].fillna(0).astype(int)
    df[['1', '2', '3']] = df[['1', '2', '3']].astype(int)
    df['label'] = df[['1', '2', '3']].idxmax(axis=1).astype(int)
    df.drop(['1', '2', '3'], axis=1, inplace=True)

    df.reset_index(drop=True, inplace=True)

    df.to_csv(f'{SAVE_PATH}/features.csv')

    counts = df['label'].value_counts()

    print("Índices en counts:", counts.index.tolist())
    print("Claves en int_to_label:", int_to_label.keys())

    print('Number of samples per class:')
    for label, number in zip(counts.index, counts.values):
        print(f'{int_to_label[label]}: {number}')

In [95]:
#subject_ids = [4, 6, 7, 8, 9, 10, 11, 13, 15, 19, 21]
subject_ids = [4, 6, 7, 8, 9, 10, 11, 13, 15, 19, 21]

for patient in subject_ids:
    print(f'Processing data for {patient}...')
    make_patient_data(patient)


Processing data for 4...
Columnas disponibles: Index(['RESP', 'PPG', 'ECG', 'EDA'], dtype='object')
Etiquetas: label
0    2407523
1    1503835
2    1205938
3     954719
Name: count, dtype: int64
Data lista para procesar: 3664492
El número de ventanas esperadas es: 4846

Ventana 0 - 30000
Ventana 750 - 30750
Ventana 1500 - 31500
Ventana 2250 - 32250
Ventana 3000 - 33000
Ventana 3750 - 33750
Ventana 4500 - 34500
Ventana 5250 - 35250
Ventana 6000 - 36000
Ventana 6750 - 36750
Ventana 7500 - 37500
Ventana 8250 - 38250
Ventana 9000 - 39000
Ventana 9750 - 39750
Ventana 10500 - 40500
Ventana 11250 - 41250
Ventana 12000 - 42000
Ventana 12750 - 42750
Ventana 13500 - 43500
Ventana 14250 - 44250
Ventana 15000 - 45000
Ventana 15750 - 45750
Ventana 16500 - 46500
Ventana 17250 - 47250
Ventana 18000 - 48000
Ventana 18750 - 48750
Ventana 19500 - 49500
Ventana 20250 - 50250
Ventana 21000 - 51000
Ventana 21750 - 51750
Ventana 22500 - 52500
Ventana 23250 - 53250
Ventana 24000 - 54000
Ventana 24750 - 54750

In [96]:
subject_ids = [4, 6, 7, 8, 9, 10, 11, 13, 15, 19, 21]

combine_files(subject_ids)
print('Processing complete.')

     HR_mean    HR_std  NN50      pNN50        TINN     rmsHRV          LF  \
0  64.799331  3.230688     9  29.032258  914.000000  39.770592  246.312945   
1  65.235118  5.630104    10  31.250000  908.090909  60.664067  275.814200   
2  64.987815  5.129443    10  31.250000  897.333333  57.220287  317.871631   
3  64.328073  3.313657     9  28.125000  914.000000  39.608976  354.793217   
4  64.290899  3.361635     8  25.806452  914.000000  38.530940  371.603869   
5  64.089312  3.182478     7  22.580645  914.000000  38.275754  402.452944   
6  63.921919  3.201196     7  22.580645  914.000000  38.783158  429.329815   
7  64.306528  5.160580     7  21.875000  963.625000  43.966630  440.258436   
8  64.277569  6.968145     6  19.354839  963.000000  63.117879  445.182266   
9  63.528930  2.906676     5  16.129032  914.000000  35.052342  447.987485   

           HF     LF_HF        sum_f  ...  Resp_E_std  Resp_IE_ratio  \
0  232.861720  1.057765  1416.924553  ...    0.776028       0.980761 

In [97]:
print('holi')

holi
